In [32]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

import submitit
import sys
from os.path import join

sys.path.append("../../src")
import simulate
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [2]:
CHROM = 1
SUMSTATS_DIR = "/u/project/pasaniuc/pasaniucdata/UKBB_IMPUTED_LD_SUMSTATS"
PLINK_PATH = join(SUMSTATS_DIR, f"genotype/raw/chr{CHROM}")
LD_PATH = join(SUMSTATS_DIR, "ld/")

In [3]:
df_params = pd.DataFrame(
    {
        "n_causal_gene": [20],
        "n_body_causal_snp": [5],
        "n_tss_causal_snp": [3],
        "prob_background_causal_snp": [0.001],
        "h2_total": [0.05],
        "h2_body": [0.03],
        "h2_tss": [0.01],
    }
)

params = df_params.iloc[0, :]

# Simulate phenotypes

In [27]:
def submit_simulate(param_i, root_dir="out/simulated_gwas"):
    import sys
    from os.path import join

    sys.path.append("../../src")
    import simulate

    params = df_params.iloc[param_i, :]
    sim = simulate.simulate(
        PLINK_PATH,
        df_gene="data/df_gene.tsv",
        n_causal_gene=int(params.n_causal_gene),
        n_body_causal_snp=int(params.n_body_causal_snp),
        n_tss_causal_snp=int(params.n_tss_causal_snp),
        prob_background_causal_snp=float(params.prob_background_causal_snp),
        h2_total=float(params.h2_total),
        h2_body=float(params.h2_body),
        h2_tss=float(params.h2_tss),
    )
    out_dir = join(root_dir, f"param_{param_i}")

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    np.save(join(out_dir, "beta.npy"), sim["beta"])
    np.save(join(out_dir, "pheno.npy"), sim["pheno"])
    np.save(join(out_dir, "beta_hat.npy"), sim["beta_hat"])

In [5]:
executor = submitit.SgeExecutor(folder="./submitit-logs")

executor.update_parameters(
    time_min=60 * 4,
    memory_g=60,
    setup=[
        "export PATH=~/project-pasaniuc/software/miniconda3/bin:$PATH",
        "export PYTHONNOUSERSITE=True",
    ],
)

jobs = executor.map_array(submit_simulate, np.arange(len(df_params)))

In [15]:
# pheno1 = np.load("simulated_gwas/pheno.npy")
# OLD_DIR = (
#     "/u/project/pasaniuc/kangchen/h2gene/analysis/simulation/out/sim/gwas/20_0.001_5_3"
# )
# pheno2 = np.load(join(OLD_DIR, "phe.npy"))

# beta_hat1 = np.load("simulated_gwas/beta_hat.npy")
# OLD_DIR = (
#     "/u/project/pasaniuc/kangchen/h2gene/analysis/simulation/out/sim/gwas/20_0.001_5_3"
# )
# beta_hat2 = np.load(join(OLD_DIR, "beta_hat.npy"))

# Partition association files

In [33]:
def submit_partition_assoc(param_i, root_dir="out/simulated_gwas"):
    with open(PLINK_PATH + ".fam") as f:
        n_indiv = len(f.readlines())
    partition = pd.read_csv("data/partition.bed", delim_whitespace=True)
    snp_info = pd.read_csv("data/snp_info.tsv", delim_whitespace=True)
    snp_info["N"] = n_indiv

    beta_hat = np.load(join(root_dir, f"param_{param_i}", "beta_hat.npy"))
    n_sim = beta_hat.shape[1]

    out_dir = join(root_dir, f"param_{param_i}", "partitioned_assoc")

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    for sim_i in range(n_sim):
        assoc = snp_info.copy()
        assoc["Z"] = np.sqrt(n_indiv) * beta_hat[:, sim_i]

        for par_i, par in partition.iterrows():
            par_snps = np.where(
                (par.CHR == assoc.CHR.values)
                & (par.START <= assoc.BP.values)
                & (assoc.BP.values < par.STOP)
            )[0]
            filename = join(out_dir, f"sim_{sim_i}_par_{par_i}.tsv.gz")
            assoc.iloc[
                par_snps,
            ].to_csv(filename, sep="\t", index=False, float_format="%.6f")

In [34]:
submit_partition_assoc(0)

# TODO: submit jobs

In [ ]:
def submit_estimate(param_i, par_i, root_dir="out/estimate"):
    
    
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
        

    . /u/local/Modules/default/init/modules.sh
    module load gcc/6.3.0
    
    {params.Rscript} {SRC_DIR}/h2gene_cli.R \
        --ld_prefix {LD_PATH}/{CHR_I}/par_{wildcards.par_i} \
        --gene_list {input.gene_list} \
        --sumstats {input.sumstats} \
        --out {output.estimate}

# estimate for each LD region
rule estimate:
    resources:
        mem_gb=lambda wildcards, attempt: attempt * 3 + 4,
        time_min=lambda wildcards, attempt: attempt * 20 if attempt <= 2 else attempt * 60
    input:
        gene_list="out/data/gene_list.bed",
        sumstats="out/sim/gwas/{sim_prefix}/sim_{sim_i}_par_{par_i}.tsv.gz"
    output:
        estimate="out/estimate/{sim_prefix}/sim_{sim_i}_par_{par_i}.rds",
    params:
        Rscript="~/project-pasaniuc/software/anaconda3/envs/r_env/bin/Rscript",
        log="log/misc.log",
        out_dir=lambda wildcards, output: output.estimate[:output.estimate.rfind('/')]
    shell:
        """


        """